In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, LSTM
from keras.utils import np_utils
import numpy as np
import glob

In [2]:
#Choice = 1 if you have access to the data but not train/test data .txt files
#Choice = 0 if you have train/test data .txt files

choice = 0

#Number of data points to take per batch
no_points = 5000

Saving 3D Arrays:
https://www.geeksforgeeks.org/how-to-load-and-save-3d-numpy-array-to-file-using-savetxt-and-loadtxt-functions/

In [3]:
#Loops through all .csv files containing normal and imbalanced data, 
#takes first 5000 points from each, assigns label in separate array
if choice == 1:
    files_no = glob.glob('normal/*.csv')
    folders_im = glob.glob('imbalance/*')
    train_data = np.empty((382,no_points,8), float)
    test_data = np.empty((382,no_points,8), float)
    i=0
    for f_on in files_no:
        source_data = np.loadtxt(f_on, delimiter=",")
        train_data[i,:,:] = source_data[0:no_points,:]
        test_data[i,:,:] = source_data[no_points:no_points*2,:]
        i=i+1

    for folder in folders_im:
        files_im = glob.glob( folder +'/*.csv')
        for f_im in files_im:
            source_data = np.loadtxt(f_im, delimiter=",")
            train_data[i,:,:] = source_data[0:no_points,:]
            test_data[i,:,:] = source_data[no_points:no_points*2,:]
            i=i+1
    
    train_reshaped = np.reshape(train_data,(train_data.shape[0],-1))
    test_reshaped = np.reshape(test_data,(train_data.shape[0],-1))
    
    np.savetxt("train_data.txt", train_reshaped)
    np.savetxt("test_data.txt", test_reshaped)
    
    train_label = np.zeros(49)
    train_label = np.append(train_label, np.ones(333), axis=0)
    test_label = np.zeros(49)
    test_label = np.append(test_label, np.ones(333), axis=0)

    np.savetxt("train_label.txt", train_label)
    np.savetxt("test_label.txt", test_label)
    
    print("Finished parsing the files")

In [4]:
#Loads train/test data from .txt files to np arrays
if choice == 0:    
    train_reload = np.loadtxt("train_data.txt")
    test_reload = np.loadtxt("test_data.txt")

    train_label = np.loadtxt('train_label.txt').astype(int)
    test_label = np.loadtxt('test_label.txt').astype(int)
    
    print(train_label.shape)
    print("Loaded data files...")

(382,)
Loaded data files...


In [5]:
train_data = np.reshape(train_reload,(train_reload.shape[0],train_reload.shape[1]//8, 8))
test_data = np.reshape(test_reload,(test_reload.shape[0],test_reload.shape[1]//8, 8))
print(train_data.shape)

(382, 5000, 8)


In [6]:
#Splits the batch of 5000 into ten batches of 50
train_data = train_data.reshape(-1,int(no_points/100),8)
test_data = test_data.reshape(-1, int(no_points/100), 8)
train_label = np.zeros(4900)
train_label = np.append(train_label, np.ones(33300), axis=0)
test_label = np.zeros(4900)
test_label = np.append(test_label, np.ones(33300), axis=0)

print(train_data.shape)

(38200, 50, 8)


In [9]:
#Build the model
model = Sequential()
model.add(LSTM(24, input_shape = (50,8), activation = 'tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(LSTM(12, activation = 'tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(LSTM(4, activation = 'tanh', recurrent_activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50, 24)            3168      
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 12)            1776      
_________________________________________________________________
lstm_5 (LSTM)                (None, 4)                 272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 5,221
Trainable params: 5,221
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_data, train_label, validation_data=(test_data, test_label), epochs=20, batch_size=500)

Epoch 1/20
77/77 [==============================] - 10s 44ms/step - loss: 0.5597 - accuracy: 0.7935 - val_loss: 0.3431 - val_accuracy: 0.8826
Epoch 2/20
77/77 [==============================] - 2s 21ms/step - loss: 0.3220 - accuracy: 0.8932 - val_loss: 0.3088 - val_accuracy: 0.8940
Epoch 3/20
77/77 [==============================] - 2s 21ms/step - loss: 0.2629 - accuracy: 0.9079 - val_loss: 0.2016 - val_accuracy: 0.9231
Epoch 4/20
77/77 [==============================] - 2s 24ms/step - loss: 0.1921 - accuracy: 0.9289 - val_loss: 0.1792 - val_accuracy: 0.9338
Epoch 5/20
77/77 [==============================] - 2s 24ms/step - loss: 0.1707 - accuracy: 0.9358 - val_loss: 0.1668 - val_accuracy: 0.9365
Epoch 6/20
77/77 [==============================] - 2s 23ms/step - loss: 0.1573 - accuracy: 0.9398 - val_loss: 0.1636 - val_accuracy: 0.9387
Epoch 7/20
77/77 [==============================] - 2s 23ms/step - loss: 0.1492 - accuracy: 0.9447 - val_loss: 0.1464 - val_accuracy: 0.9451
Epoch 8/20
7

In [9]:
test_accuracy = model.evaluate(test_data, test_label)

120/120 [==============================] - 2s 17ms/step - loss: 0.3181 - accuracy: 0.8819


In [45]:
normal_load = np.loadtxt("normal/61.44.csv", delimiter = ',')
# normal_load = np.loadtxt("imbalance/35g/56.7296.csv", delimiter = ',')
normal_data = normal_load[200000:250000,:]
print(normal_data.shape)

(50000, 8)


In [52]:
files_no = glob.glob('normal/*.csv')
# errors = numpy.empty(1)
for f_on in files_no:
    normal_data = np.loadtxt(f_on, delimiter=",")
    normal_data = normal_data.reshape(-1,50,8)
    y = model.predict_classes(normal_data)
    print(f_on, sum(y)/y.shape[0])



normal\12.288.csv [0.1494]
normal\13.1072.csv [0.1928]
normal\14.336.csv [0.2368]
normal\15.1552.csv [0.211]
normal\16.1792.csv [0.1974]
normal\17.2032.csv [0.2008]
normal\18.432.csv [0.17]
normal\19.6608.csv [0.1604]
normal\20.2752.csv [0.1376]
normal\21.7088.csv [0.179]
normal\22.3232.csv [0.1866]
normal\23.552.csv [0.1404]
normal\24.576.csv [0.139]
normal\25.6.csv [0.1254]
normal\26.624.csv [0.1596]
normal\27.4432.csv [0.2366]
normal\28.8768.csv [0.1364]
normal\29.4912.csv [0.1776]
normal\30.72.csv [0.1436]
normal\31.744.csv [0.1278]
normal\32.9728.csv [0.1218]
normal\33.5872.csv [0.1222]
normal\34.2016.csv [0.1522]
normal\35.4304.csv [0.0998]
normal\36.4544.csv [0.095]
normal\37.6832.csv [0.0956]
normal\38.2976.csv [0.0838]
normal\39.3216.csv [0.0756]
normal\40.3456.csv [0.1356]
normal\41.7792.csv [0.1234]
normal\42.3936.csv [0.1104]
normal\43.6224.csv [0.104]
normal\44.6464.csv [0.1038]
normal\45.4656.csv [0.112]
normal\46.2848.csv [0.0938]
normal\47.7184.csv [0.1138]
normal\48.94

In [10]:
#Load prediction data
pred_load = np.loadtxt("pred_data.txt")
pred_label = np.zeros(490*500)
pred_label = np.append(pred_label, np.ones(3330*500), axis=0)

In [11]:
pred_data = np.reshape(pred_load,(-1, 1, 8))
model.evaluate(pred_data, pred_label)

59688/59688 [==============================] - 197s 3ms/step - loss: 0.6386 - accuracy: 0.8145


[0.6385664343833923, 0.8144764304161072]

In [12]:
print(pred_data[0,0])

[-0.56815   0.60986   0.034358  0.033138 -0.63896   0.016935 -0.17671
  0.022027]
